# Roe Approximate Riemann Solver Shallow Water Equations

$$
\renewcommand{\v}[1]{\mathbf{#1}}
% redefine integral commands with inputs
% indefinite integral without dx
\let\oldint\int
\newcommand{\inta}[1]{\oldint\!\paren{#1}}
\newcommand{\intna}[1]{\oldint\!{#1}}
\def\int{\@ifstar{\inta}{\intna}}
% indefinite integral with dx
\newcommand{\intta}[2]{\oldint\!\paren{#1}\,\mathrm{d}{#2}}
\newcommand{\inttna}[2]{\oldint\!{#1}\,\mathrm{d}{#2}}
\def\intt{\@ifstar{\intta}{\inttna}}
% definite integral with bounds but no dx
\newcommand{\dintna}[3]{\oldint_{#1}^{#2}\!{#3}}
\newcommand{\dinta}[3]{\oldint_{#1}^{#2}\!\paren{#3}}
\def\dint{\@ifstar{\dinta}{\dintna}}
% definite integral
\newcommand{\dinttna}[4]{\oldint_{#1}^{#2}\!{#3}\,\mathrm{d}{#4}}
\newcommand{\dintta}[4]{\oldint_{#1}^{#2}\!\paren{#3}\,\mathrm{d}{#4}}
\def\dintt{\@ifstar{\dintta}{\dinttna}}
\let\oldiint\iint
\newcommand{\iinta}[1]{\oldiint\!\paren{#1}}
\newcommand{\iintna}[1]{\oldiint\!{#1}}
\def\iint{\@ifstar{\iinta}{\iintna}}
% indefinite integral with dx
\newcommand{\iintta}[2]{\oldiint\!\paren{#1}\,\mathrm{d}{#2}}
\newcommand{\iinttna}[2]{\oldiint\!{#1}\,\mathrm{d}{#2}}
\def\iintt{\@ifstar{\iintta}{\iinttna}}
% definite integral with bounds but no dx
\newcommand{\diintna}[3]{\oldiint_{#1}^{#2}\!{#3}}
\newcommand{\diinta}[3]{\oldiint_{#1}^{#2}\!\paren{#3}}
\def\diint{\@ifstar{\diinta}{\diintna}}
% definite integral
\newcommand{\diinttna}[4]{\oldiint_{#1}^{#2}\!{#3}\,\mathrm{d}{#4}}
\newcommand{\diintta}[4]{\oldiint_{#1}^{#2}\!\paren{#3}\,\mathrm{d}{#4}}
\def\diintt{\@ifstar{\diintta}{\diinttna}}
% define macro for bounds
\newcommand{\evalb}[3]{\left. #1 \right\rvert_{#2}^{#3}}
\newcommand{\evall}[2]{\left. #1 \right\rvert_{#2}}
\def\eval{\@ifstar{\evalb}{\evall}}
$$

Conservation Law
$$ \v{q}_t + \v{f}(\v{q})_x = 0 $$


In [9]:
g = var('g')
# primitive variables
h = var('h')
u = var('u')
primitive = vector([h, u])
p = primitive

# f_p(p)
flux_primitive = vector([h*u, h*u^2 + 1/2 * g * h^2])
f_p = flux_primitive

# conserved variables
q1 = var('q_1')
q2 = var('q_2')
# q = [q_1, q_2]
conserved = vector([q1, q2])
q = conserved

# transform variables
z1 = var('z_1')
z2 = var('z_2')
transform = vector([z1, z2])
z = transform

# substitution dictionaries
primitive_to_conserved = {h:q1, u: q2/q1}
p_to_q = primitive_to_conserved
primitive_to_transform = {h:z1^2, u:z2/z1}
p_to_z = primitive_to_transform
conserved_to_primitive = {q1:h, q2: h*u}
q_to_p = conserved_to_primitive
conserved_to_transform = {q1:z1^2, q2:z1*z2}
q_to_z = conserved_to_transform
transform_to_conserved = {z1:sqrt(q1), z2:q2/sqrt(q1)}
z_to_q = transform_to_conserved
transform_to_primitive = {z1:sqrt(h), z2:sqrt(h)*u}
z_to_p = transform_to_primitive


# transformation symbolic expressions
# p(q)
primitive_from_conserved = vector([q1, q2/q1])
p_q = primitive_from_conserved
# p(z)
primitive_from_transform = vector([z1^2, z2/z1])
p_z = primitive_from_transform

# q(p)
conserved_from_primitive = vector([h, h*u])
q_p = conserved_from_primitive
# q(z)
conserved_from_transform = vector([z1^2, z1*z2])
q_z = conserved_from_transform

# z(p)
transform_from_primitive = vector([sqrt(h), sqrt(h)*u])
z_p = transform_from_primitive
# z(q)
transform_from_conserved = vector([sqrt(q1), q2/sqrt(q1)])
z_q = transform_from_conserved

# transformation jacobians
p_q_j = jacobian(p_q, q)
p_z_j = jacobian(p_z, z)
q_p_j = jacobian(q_p, p)
q_z_j = jacobian(q_z, z)
z_p_j = jacobian(z_p, p)
z_q_j = jacobian(z_q, q)

flux_conserved = flux_primitive.subs(p_to_q)
f_q = flux_conserved
flux_transform = flux_primitive.subs(p_to_z)
f_z = flux_transform

# flux_jacobians
flux_primitive_jacobian = jacobian(f_p, p)
f_p_j = flux_primitive_jacobian
flux_conserved_jacobian = jacobian(f_q, q)
f_q_j = flux_conserved_jacobian
flux_transform_jacobian = jacobian(f_z, z)
f_z_j = flux_transform_jacobian

[        0         1]
[g*h - u^2       2*u]

(h, u)

In [4]:
flux_transform

(z_1*z_2, 1/2*g*z_1^4 + z_2^2)

In [5]:
flux_primitive

(h*u, 1/2*g*h^2 + h*u^2)